![nebullvm nebuly AI accelerate inference optimize DeepLearning](https://user-images.githubusercontent.com/38586138/201391643-a80407e5-2c28-409c-90c9-327795cd27e8.png)

# Accelerate Fast AI ResNet34 with Speedster

Hi and welcome 👋

In this notebook we will discover how in just a few steps you can speed up the response time of deep learning model inference using the open-source library nebullvm.

In [ ]:
%env CUDA_VISIBLE_DEVICES=0

### Fine-tune a fast.ai model


For the tutorial, we will use a fast.ai notebook for beginners in which we will classify whether the input image contains a cat (True label) or a dog (False label). Let's jump to the code.


In [ ]:
from fastai.vision.all import *

In [ ]:
path = untar_data(URLs.PETS)
files = get_image_files(path/"images")

def label_func(f): return f[0].isupper()

dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(224), num_workers=0)
dls.show_batch()

After downloading a sample of images of dogs and cats, we fine-tune the fast.ai model.







In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

In [ ]:
valid_loss, error = learn.validate()

Now that we have fine-tuned the model, let's calculate the time required to run a prediction as an average over 100 tests.


In [ ]:
import time

In [ ]:
%%capture
times = []
for _ in range(100):
    st = time.time()
    preds = learn.predict(files[0])
    times.append((time.time()-st)*1000)
fastai_vanilla_time = sum(times)/len(times)

In [ ]:
print(f"Average prediction time: {fastai_vanilla_time} ms,\nPrediction: {preds}")

In [ ]:
#learn.save(".")

### Install nebullvm

Install nebullvm:

In [ ]:
!pip install speedster

Install deep learning compilers:

In [ ]:
!python -m nebullvm.installers.auto_installer --frameworks torch --compilers all

### Data preparation

Now we prepare the dataset so that it can be processed by Speedster.

In [ ]:
import torch

xs, ys = [], []
for i, (x, y) in enumerate(dls.train):
    if i >=100:
        break
    xs.append(x)
    ys.append(y)
xs = torch.cat(xs, dim=0)
ys = torch.cat(ys, dim=0)

In [ ]:
dl_nebullvm = [((x.unsqueeze(dim=0),), y.unsqueeze(0)) for x, y in zip(xs, ys)]

In [ ]:
original_model = learn.model

### Unconstrained without accuracy loss (thus constrained)

In [ ]:
import torch
import torchvision.models as models
from speedster import optimize_model, save_model, load_model

In [ ]:
# Load a resnet as example
model = original_model

# Provide an input data for the model    
input_data = dl_nebullvm

# Run Speedster optimization
optimized_model = optimize_model(
  model, input_data=input_data, optimization_time="unconstrained",
)

# Try the optimized model
# x = torch.randn(1, 3, 224, 224)
# res = optimized_model(x)

In [ ]:
optimized_model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Set the model to eval mode and move it to the available device
model.eval()
model.to(device)

In [ ]:
res_optimized = optimized_model(x)
res_optimized

In [ ]:
from nebullvm.tools.benchmark import benchmark

benchmark(model, input_data)
benchmark(optimized_model, input_data)

### Unconstrained with 2% accuracy loss

In [ ]:
# Load a resnet as example
model = original_model

# Provide an input data for the model    
input_data = dl_nebullvm

# Run Speedster optimization
optimized_model = optimize_model(
  model, input_data=input_data, optimization_time="unconstrained", metric_drop_ths=0.02, metric="accuracy"
)

# Try the optimized model
# x = torch.randn(1, 3, 224, 224)
# res = optimized_model(x)

In [ ]:
# Set the model to eval mode and move it to the available device
model.eval()
model.to(device)

In [ ]:
optimized_model

In [ ]:
benchmark(model, input_data)
benchmark(optimized_model, input_data)

## Save and reload the optimized model

We can easily save to disk the optimized model with the following line:

In [ ]:
save_model(optimized_model, "model_save_path")

We can then load again the model:

In [ ]:
optimized_model = load_model("model_save_path")

<center> 
    <a href="https://discord.com/invite/RbeQMu886J" target="_blank" style="text-decoration: none;"> Join the community </a> |
    <a href="https://nebuly.gitbook.io/nebuly/welcome/questions-and-contributions" target="_blank" style="text-decoration: none;"> Contribute to the library </a>
</center>

<center> 
    <a href="https://github.com/nebuly-ai/nebuly/tree/main/optimization/speedster#key-concepts" target="_blank" style="text-decoration: none;"> How speedster works </a> •
    <a href="https://github.com/nebuly-ai/nebuly/tree/main/optimization/speedster#documentation" target="_blank" style="text-decoration: none;"> Documentation </a> •
    <a href="https://github.com/nebuly-ai/nebuly/tree/main/optimization/speedster#quick-start" target="_blank" style="text-decoration: none;"> Quick start </a> 
</center>